<p style="text-align:center;">
    <img src="https://raw.githubusercontent.com/skyplane-project/skyplane/main/docs/_static/logo-light-mode.png" width=500>
</p>

# Welcome to Skyplane!

## Skyplane enables fast data transfers between any cloud

Skyplane is a tool for blazingly fast bulk data transfers between object stores in the cloud. It provisions a fleet of VMs in the cloud to transfer data in parallel while using compression and bandwidth tiering to reduce cost.

Skyplane is:
1. 🔥 Blazing fast ([110x faster than AWS DataSync](https://skyplane.org/en/latest/benchmark.html))
2. 🤑 Cheap (4x cheaper than rsync)
3. 🌐 Universal (AWS, Azure and GCP)

You can use Skyplane to transfer data: 
* between object stores within a cloud provider (e.g. AWS us-east-1 to AWS us-west-2)
* between object stores across multiple cloud providers (e.g. AWS us-east-1 to GCP us-central1)
* between local storage and cloud object stores (experimental)

# Exercises

This notebook consists of 4 exercises:

1. Exercise 1: Copying data between AWS regions
2. Exercise 2: Copying data with Skyplane 
3. Exercise 3: Copying data to multiple destinations (multicast) 
4. Exercise 4: Copying data between two VMs 
5. Exercise 5: Cleanup 

# Learning outcomes 🎯

After completing this notebook, you would have:

1. An understand the Skyplane API
2. Transfered data for a ML model from AWS S3 object stores in US-EAST-1 (N. Virginia) to EU-WEST-1 (Ireland)
3. Compare and contrast `aws s3 cp` with `skyplane cp`
4. Terminate the transfer and clean up state



# How to use this Tutorial

These notebooks serve as a guide to Skyplane. At any point if you happen to get stuck, feel free to ping us on `#skyplane` channel on the [Skycamp slack.](https://join.slack.com/t/skycamp2022/shared_invite/zt-1gsrgky1z-iSFVEEOMSUD7Dd7B5syCsA)

We will describe what we are doing in this notebook. The commands and the example response are included. We highly recommend you open an terminal and run commands yourself. 

### 💻 - Run commands in an interactive terminal window

You can use this icon as a hint to know when to switch away from the current notebook and edit a file or open a terminal. We also have example outputs that you can use to ensure consistency. 


# How to open a Terminal

If you're using jupyter lab, you can create a terminal in your browser by going to `File -> New -> Terminal`

# Preflight  - Initializing cloud credentials

Before we start this tutorial, we have few pre-flight checks:

### Let's ensure we have the latest notebook

In [18]:
# Please run this cell
!git pull --quiet

There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=origin/<branch> skyplane-2023



### Update to the latest Skyplane pip package

In [22]:
!pip uninstall -y skyplane

Found existing installation: skyplane 0.3.2
Uninstalling skyplane-0.3.2:
  Successfully uninstalled skyplane-0.3.2


In [23]:
# Please run this cell
!pip install -U "git+https://github.com/skyplane-project/skyplane.git@skycamp-tutorial-2023#egg=skyplane[aws]"

DEPRECATION: git+https://github.com/skyplane-project/skyplane.git@skycamp-tutorial-2023#egg=skyplane[aws] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/skyplane-project/skyplane.git (to revision skycamp-tutorial-2023) to /private/var/folders/ps/n8nqzz915pqcc38mq5h1p7zr0000gn/T/pip-install-gvjgylgm/skyplane_d935c4c9a3d347c382baca3686b93c3b
  Running command git clone --filter=blob:none --quiet https://github.com/skyplane-project/skyplane.git /private/var/folders/ps/n8nqzz915pqcc38mq5h1p7zr0000gn/T/pip-install-gvjgylgm/skyplane_d935c4c9a3d347c382baca3686b93c3b
  Running command git checkout -b skycamp-tutorial-2023 --track origin/skycamp-tutorial-2023
  Switched to a new branch 'skycamp-tutorial-2023'
  branch 'skycamp-tutorial-2023' set up to track 'origin/skyca

  Created wheel for skyplane: filename=skyplane-0.3.2-py3-none-any.whl size=203720 sha256=7b1957b29b46f9423d66bb6751aec2d59b21ee9ccfa0f607a39c57a66865ef78
  Stored in directory: /private/var/folders/ps/n8nqzz915pqcc38mq5h1p7zr0000gn/T/pip-ephem-wheel-cache-3ys4oyqr/wheels/95/a5/1a/847a59e7211b420fa0a2dbefa6a0e8421441f6072b626e3dc2
Successfully built skyplane

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [24]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


### Configure Skyplane with AWS credentials. 

#### <span style="color:red">Choose `Y` only for AWS, and `n` for GCP and Azure.</span>

💻 `skyplane init`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/

03:37:54 [DEBUG] Found existing configuration file at /root/.skyplane/config, 
loading

(1) Configuring AWS:
    Do you want to configure AWS support in Skyplane? [Y/n]:
    Loaded AWS credentials from the AWS CLI [IAM access key ID: ...ZEXYJW]
    AWS region config file saved to /root/.skyplane/aws_config

(2) Configuring Azure:
    Do you want to configure Azure support in Skyplane? [Y/n]: n
    Disabling Azure support

(3) Configuring GCP:
    Do you want to configure GCP support in Skyplane? [Y/n]: n
    Disabling Google Cloud support

Config file saved to /root/.skyplane/config
To disable performance logging info: 
https://skyplane.org/en/latest/performance_stats_collection.html
```


> **💡 Hint** - If you run into any issues, please contact one of the Skyplane team members immediately. This step is critical to follow through the tutorial.

# Transferring Data with Skyplane

<p style="text-align:center;">
    <img src="./assets/skycamp-art.png" width=700>
</p>

The core of Skyplane is based around the `cp` command. Suppose you want to train the Stable Diffusion neural network. You’ve just gotten a fresh batch of [LAION](https://laion.ai/blog/laion-400-open-dataset/) dataset, but it is in us-east-1, while your network is in eu-west-1. Skyplane can help you efficiently transfer this data over to wherever your neural network may be so that you can train your model. Let’s prepare for a transfer by first initializing buckets in a few different cloud regions in AWS.

# Creating Buckets

### Setting up AWS in the Destination region

First, let’s create a bucket in your source AWS region to store the data for the neural network.

> **💡 Hint** - Reminder to replace [name] with a unique string. e.g., "edcvr"

💻 `aws s3 mb s3://skycamp-diffusion-data-[name] --region eu-west-1`
```
make_bucket: skycamp-diffusion-data-[name]
```

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
bucket_name = "sarajmybucket1r234"
region_tag = "aws:us-west-2"

In [26]:
import skyplane
client = skyplane.SkyplaneClient()

Logging to: /tmp/skyplane/transfer_logs/20231018_135238-afd6ef1f/client.log


In [14]:
bucket_path = client.object_store().create_bucket(region=region_tag, bucket_name=bucket_name)
bucket_path

17:58:20 [WARN]  Bucket <bound method S3Interface.bucket of <skyplane.obj_store.s3_interface.S3Interface object at 
0x13d58e650>> in region us-west-2 already exists

's3://sarajmybucket1r234'

# Exercise 1: Copying data between AWS regions

Transferring data between AWS regions with `aws s3 cp`

Each cloud provider has dedicated tools to move data between cloud regions. Let’s try transferring over the data using AWS’s built in cp command:

💻 `aws s3 cp --recursive s3://laion-400m s3://skycamp-diffusion-data-[name]`

```
copy: s3://stable-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Completed 4.2 GiB/~15.7 GiB (36.9 MiB/s) with ~12 file(s) remaining (calculating...)

```

### This will take a long time to complete. Feel free to interrupt the command. Notice that it copies data at under 25 MiB/s.


# Exercise 2: Let's try the same transfer with Skyplane

In [27]:
src_bucket_path = "s3://skycamp-demo-bucket/gorilla/"

In [28]:
bucket_path = "s3://sarajmybucket1r234"

In [29]:
client.copy(src_bucket_path, bucket_path, recursive=True, max_instances=4)

✓ Provisioning VMs (8/8) in 39.56s

✓ Starting gateway container on VMs (8/8) in 16.73s

Output()

Output()

✓ Completing multipart uploads (2/2) in 1.26s


✅ Transfer completed successfully
Transfer runtime: 14.49s, Throughput: 6.84Gbps


# Exercise 3: Transferring to multiple destinations

In [19]:
region_tag2 = "aws:us-east-1"
bucket_name2 = "fijldfjd"

In [20]:
bucket2_path = client.object_store().create_bucket(region=region_tag2, bucket_name=bucket_name2)
bucket2_path

's3://fijldfjd'

In [21]:
client.copy(src_bucket_path, [bucket_path, bucket2_path], recursive=True, max_instances=4)

✓ Provisioning VMs (12/12) in 48.52s

✓ Starting gateway container on VMs (12/12) in 20.63s

Output()

Output()

✓ Completing multipart uploads (16/16) in 6.56s

✅ Transfer completed successfully

Transfer runtime: 108.40s, Throughput: 9.06Gbps

# Exercise 4: Cleanup 

In [ ]:
client.object_store().delete_bucket(bucket_name, provider="aws")

In [16]:
client.object_store().delete_bucket(bucket_name2, provider="aws")

💻 `skyplane cp --recursive s3://laion-400m s3://skycamp-diffusion-data-[name]`

```
 _____ _   ____   _______ _       ___   _   _  _____
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __|
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/


Will transfer 32 objects totaling 35.90GB from aws:us-east-1 to aws:us-east-1
    VMs to provision: 1x aws:us-east-1
    part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    ...
a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet

Continue? [Y/n]: Y
Transfer starting (Tip: Enable auto-confirmation with `skyplane config set autoconfirm true`)

Storing debug information for transfer in /tmp/skyplane/transfer_logs/20221018_205648/client.log
✓ Initializing cloud keys (3/3) in 2.84s
✓ Provisioning gateway instances (1/1) in 34.48s
✓ Installing gateway package (1/1) in 12.41s
🚀 35.90GB transfer job launched
  Transfer progress (completing multi-part uploads) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 GiB 602.0 MB/s 0:00:00
✓ Deprovisioning instances (1/1) in 2.42s
⠙ Verifying all files were copied

✅ Transfer completed successfully
Transfer runtime: 61.94s, Throughput: 4.64Gbps

```


##  💡  Observe skyplane significantly reduces the time to move data

# Exercise 3: Let's scale up transfer bandwidth with Skyplane

💻 `skyplane cp -n 4 --recursive s3://laion-400m s3://skycamp-diffusion-data-[name]`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/


Will transfer 32 objects totaling 53.75GB from aws:us-east-1 to aws:eu-west-1
    VMs to provision: 4x aws:eu-west-1, 4x aws:us-east-1
    Estimated egress cost: $1.08 at $0.02/GB
    part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    ...
    part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Continue? [Y/n]: Y
Transfer starting (Tip: Enable auto-confirmation with `skyplane config set autoconfirm true`)

Storing debug information for transfer in /tmp/skyplane/transfer_logs/20221019_020117/client.log
✓ Initializing cloud keys (3/3) in 3.79s
✓ Provisioning gateway instances (8/8) in 43.82s
✓ Installing gateway package (8/8) in 28.26s
🚀 53.75GB transfer job launched
  Transfer progress (completing multi-part uploads) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 GiB 2.0 GB/s 0:00:00
Compression saved 0.20% of egress fees
✓ Deprovisioning instances (8/8) in 6.62s
⠸ Verifying all files were copied

✅ Transfer completed successfully
Transfer runtime: 33.09s, Throughput: 12.99Gbps
```

##  💡  Observe skyplane enables you to scale up transfer throughput by paralellizing data between multiple VMs

---

# Exercise 4: We will now try another Skyplane feature - syncing data between AWS regions
Now, let’s suppose that you have a bucket storing a backup of your dataset, and it already has some of the data there.


Let’s use Skyplane’s sync command to update the weights, and again compare the time that it takes. 


💻 `skyplane sync s3://laion-400m s3://skycamp-diffusion-data-[name]`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/

No objects need updating. Exiting...
```

---

## 💻 Terminate your cluster!

Finally, just to make sure that we don't have any instances running that might be burning up money, let's quickly deprovision everything.

💻 `skyplane deprovision`

```
No instances to deprovision
✓ Removing IPs from VPCs (4/4) in 2.05s

```

## 🎉 Congratulations! Your plane has now landed. Skyplane is an open sourced project. Feel free to use Skyplane for all your data mobility needs!


#### Eager to learn more? 

#### Feel free to play-around with the [Skyplane optimizer](https://optimizer.skyplane.org/), read our NSDI 2023 [paper](https://arxiv.org/abs/2210.07259), or browse through our GitHub [repository](https://github.com/skyplane-project/skyplane).

Acknowledgement: Thanks to [Skypilot](https://github.com/romilbhardwaj/skypilot-tutorial/) for the notebook template.

In [17]:
import time
from rich.progress import track

for i in track(range(20), description="Processing..."):
    time.sleep(1)  # Simulate work being done

Output()